In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#nltk
import nltk

#stop-words
from nltk.corpus import stopwords
stop_words=set(nltk.corpus.stopwords.words('english'))

# tokenizing
from nltk import word_tokenize,sent_tokenize

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input
from keras.models import Model

In [21]:
import pandas as pd 
import numpy as np
import string
import preprocessor as p
import re
import keras.preprocessing.text as kpt
from nltk.stem import SnowballStemmer
def clean_tweets(tweets):
        stemmer = SnowballStemmer("german")
        german_stop_words = stopwords.words('german')
        new_tweets =[]
        for tweet in tweets:
            tweet = tweet.lower()
            tweet = re.sub('[!?,.:";]', '', tweet)
            tweet = p.tokenize(tweet)
            cleaned_tweet =[];
            separator = " "
            for word in kpt.text_to_word_sequence(tweet):
                if word not in german_stop_words:
                    cleaned_tweet.append(stemmer.stem(word))
            new_tweets.append(separator.join(cleaned_tweet))
        return np.array(new_tweets)

In [22]:
corp = pd.read_csv('training.csv')
labels = corp[['x_coord','y_coord']]
labels = np.asarray(labels).astype('float32')
corp = corp['tweet']
corp = np.array(corp)
corp = clean_tweets(corp)
print(corp.shape)
no_docs=len(corp)

(22583,)


In [23]:
vocab_size=500 
encod_corp=[]
for i,doc in enumerate(corp):
    encod_corp.append(one_hot(doc,vocab_size))
    if i < 10 : 
        print("The encoding for document",i+1," is : ",one_hot(doc,50))

The encoding for document 1  is :  [4, 14, 11, 23, 2, 41, 13, 20, 20, 15, 7, 36, 43, 8, 38, 23, 2, 41, 13, 20, 25, 48, 44, 3, 9, 44, 40, 12, 44, 30, 17, 23, 40, 30, 27, 42, 20, 43, 4, 29, 46, 41, 47, 20, 44, 31, 26, 43, 47, 40]
The encoding for document 2  is :  [37, 44, 18, 43, 16, 44, 16, 25, 30, 20, 30, 39, 8, 18, 4, 30, 34, 4, 2, 10, 33, 25, 38, 41, 3, 22, 39, 14, 15, 7, 44, 30, 22, 28, 27, 22, 40, 49, 11, 42, 28, 43, 31, 38, 27, 36, 39, 4, 38, 24, 22, 6, 42, 29, 33, 30, 30, 34, 15, 29, 7, 13, 43, 37, 31, 49, 38, 34, 2, 34, 46, 20, 36, 29, 34, 2, 40, 35, 29, 28, 15, 32, 34, 32, 29, 21, 7, 34, 42, 33, 30]
The encoding for document 3  is :  [43, 14, 22, 5, 33, 27, 40, 11, 34, 48, 3, 31, 47, 42, 43, 42, 43, 26, 49, 9, 43, 44, 42, 42, 27, 43, 44, 42]
The encoding for document 4  is :  [42, 28, 21, 41, 46, 35, 4]
The encoding for document 5  is :  [40, 15, 15, 14, 30, 1, 21, 9, 25, 36, 2, 44, 15, 38, 20, 4, 34, 43, 23, 12, 1, 43, 16, 4, 31, 46, 8, 4, 14, 45, 27, 13, 30, 30, 12, 35, 9, 1

In [24]:
maxlen=-1
for doc in corp:
    tokens=nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen=len(tokens)
maxlen = 163
print("The maximum number of words in any document is : ",maxlen)


The maximum number of words in any document is :  163


In [25]:
pad_corp=pad_sequences(encod_corp,maxlen=maxlen,padding='post',value=0.0)
print("No of padded documents: ",len(pad_corp))

No of padded documents:  22583


In [26]:
for i,doc in enumerate(pad_corp):
    if i < 10:
        print("The padded encoding for document",i+1," is : ",doc)

The padded encoding for document 1  is :  [368 388 494 104 461  84 339 170 451 233 462 482 208  17 498 104 461  84
 339 170 427 308  63 480 154 495 222 352  66 367  98 376 275 367 316 305
 375 208  28 297 108 335  13 451  66  28 494 437  13  23   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]
The padded encoding for document 2  is :  [ 47  66 231 208 208  66  74 184 367  60 367 469 361 231 193 367 273 193
  88 268  78 143 155  28 480 411 469 241 470 482 495 367   7 366  44  84
  17 398 270 447 210  51 272 155 248 111 469 443 155 368 249 300 447 276
 204 367 367 273 154 297  23 339 2

In [38]:
input=Input(shape=(no_docs,maxlen),dtype='float64')

In [39]:
word_input=Input(shape=(maxlen,),dtype='float64')  

# creating the embedding
word_embedding=Embedding(input_dim=vocab_size,output_dim=8,input_length=maxlen)(word_input)

word_vec=Flatten()(word_embedding) # flatten
embed_model =Model([word_input],word_vec) # combining all into a Keras model

In [40]:
embed_model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),loss='binary_crossentropy',metrics=['acc']) 
# compiling the model. parameters can be tuned as always.

In [41]:
embeddings=embed_model.predict(pad_corp) # finally getting the embeddings.

In [42]:
print("Shape of embeddings : ",embeddings.shape)
print(embeddings)

Shape of embeddings :  (22583, 1304)
[[ 0.0014898   0.02367235 -0.02727101 ...  0.04181394  0.03304669
   0.00461565]
 [ 0.04116328 -0.04411019  0.02400515 ...  0.04181394  0.03304669
   0.00461565]
 [-0.01254798 -0.02896338  0.00571003 ...  0.04181394  0.03304669
   0.00461565]
 ...
 [ 0.0465141  -0.00689527  0.02573087 ...  0.04181394  0.03304669
   0.00461565]
 [ 0.04504078 -0.00161718 -0.00223484 ...  0.04181394  0.03304669
   0.00461565]
 [-0.00362154  0.03492601  0.01847563 ...  0.04181394  0.03304669
   0.00461565]]


In [32]:
embeddings=embeddings.reshape(-1,maxlen,8)
print("Shape of embeddings : ",embeddings.shape) 
print(embeddings)

Shape of embeddings :  (3044, 163, 8)
[[[ 0.00737191 -0.01072978  0.04837361 ... -0.04897338  0.04107499
   -0.02110729]
  [ 0.01007394  0.01935105  0.03972271 ...  0.044522   -0.00850205
   -0.01058699]
  [-0.02017479 -0.00364791  0.0089342  ... -0.03343781  0.01522208
   -0.03107841]
  ...
  [-0.04654595  0.03871806 -0.00667898 ...  0.04821194  0.04132337
   -0.04238854]
  [-0.04654595  0.03871806 -0.00667898 ...  0.04821194  0.04132337
   -0.04238854]
  [-0.04654595  0.03871806 -0.00667898 ...  0.04821194  0.04132337
   -0.04238854]]

 [[ 0.02506054  0.01809522 -0.01266562 ... -0.00267894 -0.04480786
   -0.02374984]
  [-0.03563181 -0.0059803   0.00458474 ...  0.03748833 -0.01929357
   -0.02354547]
  [ 0.00561469  0.02385304 -0.03038219 ...  0.01152345 -0.04293685
    0.04764937]
  ...
  [-0.04654595  0.03871806 -0.00667898 ...  0.04821194  0.04132337
   -0.04238854]
  [-0.04654595  0.03871806 -0.00667898 ...  0.04821194  0.04132337
   -0.04238854]
  [-0.04654595  0.03871806 -0.00667

In [43]:
new_embeddings = []
for embed in embeddings:
    new_embeddings.append(embed.flatten())
new_embeddings = np.array(new_embeddings)
print(new_embeddings.shape)

(22583, 1304)


In [44]:
print(labels.shape)

(22583, 2)


In [20]:
from sklearn.linear_model import LinearRegression
LinearRegressionModel = LinearRegression()
LinearRegressionModel.fit(new_embeddings,labels)

LinearRegression()

In [37]:
from sklearn.metrics import mean_squared_error
y_pred = LinearRegressionModel.predict(new_embeddings)
print(y_pred[0])
print(mean_squared_error(labels, y_pred))

[51.653076   9.5217285]
1.2488022
